#Importing Functions

In [1]:
!pip install tensorflow_model_optimization

In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master')
import tensorflow_model_optimization as tfmot
from qkeras import *


In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from scipy import ndimage
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.layers import BatchNormalization,ReLU,GlobalAveragePooling1D,MaxPooling1D,LSTM,TimeDistributed,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.models import save_model, load_model
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow.keras.utils import  plot_model
from keras.models import Model,save_model,load_model
from keras.layers import Input
from keras.layers.merge import concatenate
import numpy as np
from six.moves import zip
from tensorflow.keras import callbacks
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from qkeras import QActivation
from qkeras import QDense
from qkeras import QConv2D
from qkeras import quantized_bits
from qkeras.utils import load_qmodel
from qkeras.utils import print_model_sparsity
from tensorflow_model_optimization.python.core.sparsity.keras import prune
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_schedule
from qkeras.estimate import print_qstats
from datetime import datetime
from packaging import version

import os
import tempfile
import os
import tensorflow as tf
from tensorflow import expand_dims
from tensorflow import keras
%load_ext tensorboard

In [4]:
!pip install -U tensorboard_plugin_profile

Requirement already up-to-date: tensorboard_plugin_profile in /usr/local/lib/python3.6/dist-packages (2.3.0)


#Loading Dataset

In [5]:
def load_file(filepath):
  dataframe=read_csv(filepath,header=None,delim_whitespace=True)
  return dataframe.values

In [6]:
def load_group(filenames,prefix=''):
  loaded=list()
  for name in filenames:
    data=load_file(prefix+name)
    loaded.append(data)
  loaded=dstack(loaded)
  return loaded

In [7]:
def load_dataset_group(group,prefix=''):
  filepath=prefix+group+'/Inertial Signals/'
  filenames=list()
  filenames+=['total_acc_x_'+group+'.txt','total_acc_y_'+group+'.txt','total_acc_z_'+group+'.txt']
  filenames+=['body_acc_x_'+group+'.txt','body_acc_y_'+group+'.txt','body_acc_z_'+group+'.txt']
  filenames+=['body_gyro_x_'+group+'.txt','body_gyro_y_'+group+'.txt','body_gyro_z_'+group+'.txt']
  X=load_group(filenames,filepath)
  y=load_file(prefix+group+'/y_'+group+'.txt')
  return (X,y)


In [8]:
def load_dataset(prefix=''):
  trainX,trainy=load_dataset_group('train',prefix+'/content/drive/My Drive/Colab/MS Project/UCI HAR Dataset/UCI HAR Dataset/')
  print(trainX.shape,trainy.shape)
  testX,testy=load_dataset_group('test',prefix+'/content/drive/My Drive/Colab/MS Project/UCI HAR Dataset/UCI HAR Dataset/')
  print(testX.shape,testy.shape)
  trainy=trainy-1
  testy=testy-1
  trainy=to_categorical(trainy)
  testy=to_categorical(testy)
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy

In [9]:
def scale_data(trainX, testX):
	# remove overlap
	cut = int(trainX.shape[1] / 2)
	longX = trainX[:, -cut:, :]
	# flatten windows
	longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
	# flatten train and test
	flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
	flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
	# standardize

	s = StandardScaler()
		# fit on training data
	s.fit(longX)
		# apply to training and test data
	longX = s.transform(longX)
	flatTrainX = s.transform(flatTrainX)
	flatTestX = s.transform(flatTestX)
	# reshape
	flatTrainX = flatTrainX.reshape((trainX.shape))
	flatTestX = flatTestX.reshape((testX.shape))
	return flatTrainX, flatTestX

##Function for model size

In [10]:
def get_zipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  zipped_file = file+'.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [11]:
optimizer=Adam(lr=0.0001)
with_bn=1
THRESHOLD=0.1

class LearningRateAdjuster(callbacks.Callback):
  def __init__(self):
    self.learning_rate_factor = 1.0
    pass

  def on_epoch_end(self, epochs, logs):
    max_variance = -1

    for layer in self.model.layers:
      if layer.__class__.__name__ in [
          "BatchNormalization",
          "QBatchNormalization"
      ]:
        variance = np.max(layer.get_weights()[-1])
        if variance > max_variance:
          max_variance = variance

    if max_variance > 32 and self.learning_rate_factor < 100:
      learning_rate = K.get_value(self.model.optimizer.learning_rate)
      self.learning_rate_factor /= 2.0
      print("***** max_variance is {} / lr is {} *****".format(
          max_variance, learning_rate))
      K.eval(K.update(
          self.model.optimizer.learning_rate, learning_rate / 2.0
      ))

lra = LearningRateAdjuster()

##1. CNN MODEL


In [28]:
trainX,trainy,testX,testy=load_dataset()
verbose,epochs,batch_size=1,50,32
n_timesteps,n_features,n_outputs=trainX.shape[1],trainX.shape[2],trainy.shape[1]
print('n step: ', n_timesteps)
print('features: ',n_features)
trainX,testX=scale_data(trainX,testX)
print(trainX.shape)
prune_whole_model=True

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
n step:  128
features:  9
(7352, 128, 9)


In [29]:

'''

inputs=keras.Input(shape=(n_timesteps,n_features))


conv_1=tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu')(inputs)
maxpool_1=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(conv_1)

conv_2=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu')(maxpool_1)
maxpool_2=tf.keras.layers.MaxPooling1D(pool_size=2,strides=1)(conv_2)

conv_3=tf.keras.layers.Conv1D(filters=32,kernel_size=3,strides=1,activation='relu')(maxpool_2)
avg_pooling=tf.keras.layers.GlobalAveragePooling1D()(conv_3)
batch_norm=tf.keras.layers.BatchNormalization()(avg_pooling)

output=tf.keras.layers.Dense(n_outputs,activation='softmax')(batch_norm)
model=tf.keras.Model(inputs=inputs,outputs=output)
'''
def build_model(n_timesteps,n_features):
  x=x_in=Input(shape=(n_timesteps,n_features),name='input')
  x=QActivation("quantized_relu_po2(4,1)",name="acti")(x)
  x=QConv1D(filters=64, kernel_size=3,
      strides=1,
      kernel_quantizer=quantized_po2(4, 1),
      bias_quantizer=quantized_bits(4,2,0) if not with_bn else None,
      bias_range=4 if not with_bn else None,
      use_bias=not with_bn,
      name="conv1d_0_m")(x)
  x = QActivation("quantized_relu(3,1)", name="act0_m")(x)
  x=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(x)
  x=QConv1D(filters=128, kernel_size=3,
      strides=1,
      kernel_quantizer=quantized_po2(4, 1),
      bias_quantizer=quantized_bits(4,2,1) if not with_bn else None,
      bias_range=4 if not with_bn else None,
      use_bias=not with_bn,
      name="conv1d_1_m")(x)
  x = QActivation("quantized_relu(3,1)", name="act1_m")(x)
  x=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(x)
  x=QConv1D(filters=32, kernel_size=3,
      strides=1,
      kernel_quantizer=quantized_po2(4, 1),
      bias_quantizer=quantized_bits(4,2,2) if not with_bn else None,
      bias_range=4 if not with_bn else None,
      use_bias=not with_bn,
      name="conv1d_2_m")(x)
  x = QActivation("quantized_relu(3,1)", name="act2_m")(x)
  x=tf.keras.layers.GlobalAveragePooling1D()(x)
  if with_bn:
    x=QBatchNormalization(
        gamma_quantizer=quantized_relu_po2(4,8),
        variance_quantizer=quantized_relu_po2(6),
        beta_quantizer=quantized_po2(4, 4),
        gamma_range=8,
        beta_range=4,
        name="bn1")(x)
  x = QActivation("quantized_relu(3,1)", name="act3_m")(x)
  x = Flatten()(x)
  x = QDense(
      n_outputs,
      kernel_quantizer=quantized_ulaw(4, 0, 1),
      bias_quantizer=quantized_bits(4, 0, 1),
      name="dense")(
          x)
  x = Activation("softmax", name="softmax")(x)
  model = Model(inputs=[x_in], outputs=[x])
  return model

In [30]:
def train_and_save(model,trainX,trainy,testX,testy):

  plot_model(model, show_shapes=True, to_file='/content/drive/My Drive/Colab/MS Project/My Model/CNN_Model.png')
  model.compile(
      loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  model.summary()
  callbacks = [
        pruning_callbacks.UpdatePruningStep(),
        #pruning_callbacks.PruningSummaries(log_dir=tempfile.mkdtemp())
        pruning_callbacks.PruningSummaries(log_dir="mnist_prune")
    ]
  model.fit(
      trainX, trainy, batch_size=batch_size,
      epochs=epochs, initial_epoch=1, verbose=1,
      validation_data=(testX,testy),
      callbacks=callbacks)
  score = model.evaluate(testX, testy, verbose=False)
  print("Test score:", score[0])
  print("Test accuracy:", score[1])
  print_model_sparsity(model)
  keras_file='pruned_model.h5'
  print("Saving mode to: ",keras_file)
  save_model(model,keras_file)
  print("Reloading model")
  with prune.prune_scope():
      loaded_model = load_qmodel(keras_file)
  score = loaded_model.evaluate(testX, testy, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])

  return keras_file,model

In [31]:
batch_size=batch_size
epoch=5
validation_split=0.3
num=trainX.shape[0]*(1-validation_split)
pruning_params = {
        "pruning_schedule":
            tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1, final_sparsity=0.9,begin_step=0,end_step=np.ceil(num / batch_size).astype(np.int32) * 5)
    }
    
if prune_whole_model:
      model = build_model(n_timesteps,n_features)
      model = prune.prune_low_magnitude(model, **pruning_params)
else:
      model = build_layerwise_model(input_shape, **pruning_params)

keras_file,model=train_and_save(model, trainX, trainy, testX, testy)

/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/qkeras/qnormalization.py:81: UserWarning: gamma_range is deprecated in QBatchNormalization layer.
  warnings.warn('gamma_range is deprecated in QBatchNormalization layer.')
/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/qkeras/qnormalization.py:84: UserWarning: beta_range is deprecated in QBatchNormalization layer.
  warnings.warn('beta_range is deprecated in QBatchNormalization layer.')


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 9)]          0         
_________________________________________________________________
prune_low_magnitude_acti (Pr (None, 128, 9)            1         
_________________________________________________________________
prune_low_magnitude_conv1d_0 (None, 126, 64)           3458      
_________________________________________________________________
prune_low_magnitude_act0_m ( (None, 126, 64)           1         
_________________________________________________________________
prune_low_magnitude_max_pool (None, 63, 64)            1         
_________________________________________________________________
prune_low_magnitude_conv1d_1 (None, 61, 128)           49154     
_________________________________________________________________
prune_low_magnitude_act1_m ( (None, 61, 128)          

230/230 [==============================] - 5s 20ms/step - loss: 0.4767 - accuracy: 0.8347 - val_loss: 0.7881 - val_accuracy: 0.7072
Epoch 3/50
230/230 [==============================] - 4s 16ms/step - loss: 0.4749 - accuracy: 0.8380 - val_loss: 0.8394 - val_accuracy: 0.6973
Epoch 4/50
230/230 [==============================] - 4s 16ms/step - loss: 0.7027 - accuracy: 0.7856 - val_loss: 0.7688 - val_accuracy: 0.7608
Epoch 5/50
230/230 [==============================] - 4s 16ms/step - loss: 0.7779 - accuracy: 0.7454 - val_loss: 0.7736 - val_accuracy: 0.8392
Epoch 6/50
230/230 [==============================] - 4s 16ms/step - loss: 0.7052 - accuracy: 0.8439 - val_loss: 0.7584 - val_accuracy: 0.7923
Epoch 7/50
230/230 [==============================] - 4s 16ms/step - loss: 0.6613 - accuracy: 0.8596 - val_loss: 0.7136 - val_accuracy: 0.8290
Epoch 8/50
230/230 [==============================] - 4s 16ms/step - loss: 0.6252 - accuracy: 0.8682 - val_loss: 0.6503 - val_accuracy: 0.8347
Epoch 9/50

In [32]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 )
model.summary()


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 9)]          0         
_________________________________________________________________
prune_low_magnitude_acti (Pr (None, 128, 9)            1         
_________________________________________________________________
prune_low_magnitude_conv1d_0 (None, 126, 64)           3458      
_________________________________________________________________
prune_low_magnitude_act0_m ( (None, 126, 64)           1         
_________________________________________________________________
prune_low_magnitude_max_pool (None, 63, 64)            1         
_________________________________________________________________
prune_low_magnitude_conv1d_1 (None, 61, 128)           49154     
_________________________________________________________________
prune_low_magnitude_act1_m ( (None, 61, 128)          

In [33]:
model_size=get_zipped_model_size(keras_file)
print(" Size of pruned qkeras cnn model: {:.2f}KB".format(model_size/1000))

 Size of pruned qkeras cnn model: 298.67KB


In [34]:
model=sparsity.strip_pruning(model)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 9)]          0         
_________________________________________________________________
acti (QActivation)           (None, 128, 9)            0         
_________________________________________________________________
conv1d_0_m (QConv1D)         (None, 126, 64)           1728      
_________________________________________________________________
act0_m (QActivation)         (None, 126, 64)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 63, 64)            0         
_________________________________________________________________
conv1d_1_m (QConv1D)         (None, 61, 128)           24576     
_________________________________________________________________
act1_m (QActivation)         (None, 61, 128)          

In [35]:
for i, w in enumerate(model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

conv1d_0_m/kernel:0 -- Total:1728, Zeros: 89.99%
conv1d_1_m/kernel:0 -- Total:24576, Zeros: 90.00%
conv1d_2_m/kernel:0 -- Total:12288, Zeros: 90.00%
bn1/gamma:0 -- Total:32, Zeros: 0.00%
bn1/beta:0 -- Total:32, Zeros: 0.00%
bn1/moving_mean:0 -- Total:32, Zeros: 0.00%
bn1/moving_variance:0 -- Total:32, Zeros: 0.00%
dense/kernel:0 -- Total:192, Zeros: 90.10%
dense/bias:0 -- Total:6, Zeros: 0.00%
